In [1]:
"""
Task 15 - LangGraph Day 3
Basic Customer Support Bot (till Part 3)
LLM: Ollama (llama3)
"""

from typing import TypedDict

from langgraph.graph import StateGraph, END
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate


# -------------------------------------------------
# Define State
# -------------------------------------------------
class SupportState(TypedDict):
    query: str
    category: str
    response: str


# -------------------------------------------------
# Create LLM
# -------------------------------------------------
llm = ChatOllama(
    model="llama3",
    temperature=0
)


# -------------------------------------------------
# Classification Prompt
# -------------------------------------------------
classification_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a customer support classifier.\n"
            "Classify the query into ONE category only:\n"
            "- order\n"
            "- refund\n"
            "- general\n\n"
            "Respond with only the category name."
        ),
        ("human", "{query}")
    ]
)


# -------------------------------------------------
# Classifier Node
# -------------------------------------------------
def classify_query(state: SupportState) -> SupportState:
    chain = classification_prompt | llm
    category = chain.invoke(
        {"query": state["query"]}
    ).content.strip().lower()

    return {
        **state,
        "category": category
    }


# -------------------------------------------------
# Order Support Node
# -------------------------------------------------
def handle_order(state: SupportState) -> SupportState:
    return {
        **state,
        "response": (
            "For order-related issues, please provide your order ID. "
            "Our team will assist you shortly."
        )
    }


# -------------------------------------------------
# Refund Support Node
# -------------------------------------------------
def handle_refund(state: SupportState) -> SupportState:
    return {
        **state,
        "response": (
            "Refunds are processed within 5–7 business days. "
            "Please share your order ID to proceed."
        )
    }


# -------------------------------------------------
# General Support Node
# -------------------------------------------------
def handle_general(state: SupportState) -> SupportState:
    return {
        **state,
        "response": (
            "Thank you for reaching out! "
            "Our support team is happy to help with your query."
        )
    }


# -------------------------------------------------
# Router Function
# -------------------------------------------------
def route_query(state: SupportState) -> str:
    if state["category"] == "order":
        return "order"
    if state["category"] == "refund":
        return "refund"
    return "general"


# -------------------------------------------------
# Build LangGraph
# -------------------------------------------------
graph = StateGraph(SupportState)

graph.add_node("classifier", classify_query)
graph.add_node("order", handle_order)
graph.add_node("refund", handle_refund)
graph.add_node("general", handle_general)

graph.set_entry_point("classifier")

graph.add_conditional_edges(
    "classifier",
    route_query,
    {
        "order": "order",
        "refund": "refund",
        "general": "general"
    }
)

graph.add_edge("order", END)
graph.add_edge("refund", END)
graph.add_edge("general", END)

app = graph.compile()


# -------------------------------------------------
# Run Bot
# -------------------------------------------------
if __name__ == "__main__":
    queries = [
        "Where is my order?",
        "I want a refund for my purchase",
        "Do you offer international shipping?"
    ]

    for q in queries:
        result = app.invoke({"query": q})
        print("\nCustomer Query:", q)
        print("Category:", result["category"])
        print("Bot Response:", result["response"])



Customer Query: Where is my order?
Category: order
Bot Response: For order-related issues, please provide your order ID. Our team will assist you shortly.

Customer Query: I want a refund for my purchase
Category: refund
Bot Response: Refunds are processed within 5–7 business days. Please share your order ID to proceed.

Customer Query: Do you offer international shipping?
Category: general
Bot Response: Thank you for reaching out! Our support team is happy to help with your query.
